In [1]:
import time, torch, os, cv2
from deltacnn import DCConv2d, DCBackend, DCThreshold
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.layers import ShapeSpec
from detectron2.modeling import build_resnet_backbone

In [6]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=False)

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to C:\Users\aminp/.cache\torch\hub\v0.10.0.zip


In [11]:
from ptflops import get_model_complexity_info

with torch.cuda.device(0):
  net = model
  macs, params = get_model_complexity_info(net, (3, 800, 1365), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
  print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
  print('{:<30}  {:<8}'.format('Number of parameters: ', params))

MaskRCNN(
  63.119 M, 100.000% Params, 281.578 GMac, 100.000% MACs, 
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    45.517 M, 72.113% Params, 228.407 GMac, 81.117% MACs, 
    (body): IntermediateLayerGetter(
      42.172 M, 66.814% Params, 167.462 GMac, 59.473% MACs, 
      (conv1): Conv2d(0.0 M, 0.000% Params, 2.529 GMac, 0.898% MACs, 3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(0.0 M, 0.000% Params, 0.017 GMac, 0.006% MACs, inplace=True)
      (maxpool): MaxPool2d(0.0 M, 0.000% Params, 0.017 GMac, 0.006% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        0.0 M, 0.000% Params, 14.39 GMac, 5.111% MACs, 
        (0): Bottleneck(
          0.0 M, 0.000% Params, 4.98 GMac, 1.769% MA

In [10]:
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models.detection import MaskRCNN

backbone = resnet_fpn_backbone('resnet101', False)
model = MaskRCNN(backbone, 91)

In [ ]:
model = torch.load("delta_mask-RCNN_resnet101.pth", map_location='cpu')
DCConv2d.backend = DCBackend.deltacnn

In [2]:
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, images, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.images = images
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        # image = read_image(img_path)
        image = cv2.imread(img_path)
        image = cv2.resize(image, dsize=(560, 560))
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image

In [3]:
img_dir = 'D:/Samsung/Delta-Detectron2/MVI_39051'
images = os.listdir(img_dir)[:200]
dataset = CustomImageDataset(img_dir, images)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [ ]:
start = time.time()
with torch.cuda.amp.autocast():
    for img in tqdm(dataset):
        out = model(img)
print(time.time()-start)

In [4]:
from resnet import build_delta_resnet_backbone
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_C4_3x.yaml"))
# cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_C4_1x.yaml"))
cfg.MODEL.WEIGHTS = "D:/Samsung/Delta-Detectron2/faster-rcnn_resnet50.pkl"

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.DEVICE='cuda'

DCThreshold.t_default=1.2

In [ ]:
resnet = build_resnet_backbone(cfg, input_shape=ShapeSpec(channels=3)).to('cuda')
start = time.time()
for img in tqdm(dataloader):
    img = img.to('cuda').float()
    img = img.permute(0, 3, 1, 2)
    out = resnet(img)
print(time.time()-start)

In [ ]:
resnet = build_resnet_backbone(cfg, input_shape=ShapeSpec(channels=3)).to('cuda')
for img in dataloader:
    print(img.shape)
    img = img.to('cuda').float()
    img = img.permute(0, 3, 1, 2)
    out = resnet(img.to('cuda'))
    print(out['res4'].shape)
    break

In [ ]:
print(resnet)

In [5]:
resnet = build_delta_resnet_backbone(cfg, input_shape=ShapeSpec(channels=3)).to('cuda')
flag=False
for img in dataloader:
    # print(img.shape)
    img = img.to('cuda').float()
    img = img.permute(0, 3, 1, 2)
    out = resnet(img)
    # print(out['res4'].shape)
    print("\n\n\n")
    if flag:
        break
    flag = True


stem: 
torch.Size([1, 3, 560, 560])
tensor(313600, device='cuda:0')
torch.Size([1, 64, 280, 280]) torch.Size([3, 7, 7, 64])
tensor(78400, device='cuda:0')
torch.Size([1, 64, 140, 140])
tensor(19600, device='cuda:0')



 res2
init torch.Size([1, 64, 140, 140])
tensor(19600, device='cuda:0')
conv1 torch.Size([1, 64, 140, 140]) torch.Size([64, 1, 1, 64])
tensor(19600, device='cuda:0')
conv2 torch.Size([1, 64, 140, 140]) torch.Size([64, 3, 3, 64])
tensor(19600, device='cuda:0')
conv3 torch.Size([1, 64, 140, 140]) torch.Size([64, 1, 1, 256])
tensor(19600, device='cuda:0')
shortcut torch.Size([1, 64, 140, 140]) torch.Size([64, 1, 1, 256])
tensor(19600, device='cuda:0')
init torch.Size([1, 256, 140, 140])
tensor(19600, device='cuda:0')
conv1 torch.Size([1, 256, 140, 140]) torch.Size([256, 1, 1, 64])
tensor(19600, device='cuda:0')
conv2 torch.Size([1, 256, 140, 140]) torch.Size([64, 3, 3, 64])
tensor(19600, device='cuda:0')
conv3 torch.Size([1, 256, 140, 140]) torch.Size([64, 1, 1, 256])
tens

In [ ]:
print(resnet)

In [ ]:
resnet = build_delta_resnet_backbone(cfg, input_shape=ShapeSpec(channels=3)).to('cuda')
start = time.time()
for img in tqdm(dataloader):
    img = img.to('cuda').float()
    img = img.permute(0, 3, 1, 2)
    out = resnet(img)
print(time.time()-start)

In [ ]:
from ptflops import get_model_complexity_info

with torch.cuda.device(0):
  net = build_resnet_backbone(cfg, input_shape=ShapeSpec(channels=3))
  macs, params = get_model_complexity_info(net, (3, 224, 224), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
  print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
  print('{:<30}  {:<8}'.format('Number of parameters: ', params))